In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import pandas as pd
import nltk
import nltk.sentiment
from nltk.corpus import stopwords
from wordcloud import WordCloud
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from requests import get
from bs4 import BeautifulSoup
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import re
import time
import acquire
import prepare
import explore
from env import github_token, github_username

In [2]:
def split_minecraft_data(df):
    '''
    This function performs split on minecraft repo data, stratified on language.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.language)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.language)
    return train, validate, test

#call it with: train, validate, test = split_minecraft_data(df)

In [3]:
df = pd.read_csv('clean_scraped_data.csv', index_col=[0])

In [4]:
df=df.dropna()

In [5]:
df

,repo,language,readme_contents,clean,lemmatized
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...,minecraft simple minecraftinspired demo writte...,minecraft simple minecraftinspired demo writte...
2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...,docker pullshttpsimgshieldsiodockerpullsitzgmi...,docker pullshttpsimgshieldsiodockerpullsitzgmi...
3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...,minecraft overviewer build status andrew brown...,minecraft overviewer build status andrew brown...
4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...,minecraftbyexample 1164 purpose minecraftbyexa...,minecraftbyexample 1164 purpose minecraftbyexa...
5,minecraft-dev/MinecraftDev,Kotlin,"<p align=""center""><a href=""https://minecraftde...",p aligncentera hrefhttpsminecraftdevorgimg src...,p aligncentera hrefhttpsminecraftdevorgimg src...
...,...,...,...,...,...
994,Crossroads-Development/Crossroads,Java,# Crossroads\n\nA steampunk technology mod for...,crossroads steampunk technology mod minecraft ...,crossroad steampunk technology mod minecraft i...
995,GeyserMC/PacketLib,Java,# PacketLib\nPacketLib is a library for packet...,packetlib packetlib library packetbased networ...,packetlib packetlib library packetbased networ...
996,Nic4Las/Minecraft-Enderite-Mod,Java,# Enderite Mod\n\n[![](http://cf.way2muchnoise...,enderite mod httpcfway2muchnoiseeuversions3992...,enderite mod httpcfway2muchnoiseeuversions3992...
998,espertus/blockly-lua,JavaScript,Blockly Lua allows you to program ComputerCraf...,blockly lua allows program computercraft turtl...,blockly lua allows program computercraft turtl...


In [6]:
df.language.value_counts()

Java                 402
JavaScript            86
Python                76
C#                    43
C++                   36
Shell                 35
PHP                   33
Kotlin                23
TypeScript            22
Go                    19
C                     16
Rust                  15
Lua                    6
HTML                   6
ZenScript              5
Ruby                   5
CSS                    4
GLSL                   4
Dockerfile             4
Scala                  4
Jupyter Notebook       4
Batchfile              3
CoffeeScript           3
HCL                    3
Elixir                 2
mcfunction             2
Visual Basic .NET      2
TeX                    2
Mustache               2
PowerShell             2
GDScript               1
R                      1
CMake                  1
Vue                    1
AutoHotkey             1
SCSS                   1
PureBasic              1
Swift                  1
JSON                   1
Assembly               1


In [7]:
df.language.unique()

array(['Python', 'Shell', 'Java', 'Kotlin', 'C', 'C#', 'Lua', 'C++',
       'JavaScript', 'PHP', 'TypeScript', 'SystemVerilog', 'Scala',
       'PowerShell', 'CSS', 'Batchfile', 'Mustache', 'Rust', 'Go',
       'CoffeeScript', 'Dockerfile', 'Clojure', 'PureBasic', 'TeX',
       'Jupyter Notebook', 'Smarty', 'HTML', 'Yacc', 'Ruby', 'mcfunction',
       'HCL', 'Assembly', 'ZenScript', 'GLSL', 'JSON', 'Swift', 'Vue',
       'SCSS', 'AutoHotkey', 'Visual Basic .NET', 'Elixir', 'GDScript',
       'CMake', 'R', 'Haskell'], dtype=object)

In [ ]:
df["readme_length"]= df["readme_contents"].str.len()

In [ ]:
df["lemma_length"]= df["lemmatized"].str.len()

In [ ]:
df["clean_length"]= df["clean"].str.len()

In [ ]:
df.head()

In [ ]:
df.readme_length.unique()

In [ ]:
df.readme_length.max()

In [ ]:
df["java"]=df.loc[df['language'] == 'Java']

In [ ]:
 'Shell', 'Kotlin', 'C', 'C#', 'Lua', 'C++',
       'PHP', 'TypeScript', 'SystemVerilog', 'Scala',
       'PowerShell', 'CSS', 'Batchfile', 'Mustache', 'Rust', 'Go',
       'CoffeeScript', 'Dockerfile', 'Clojure', 'PureBasic', 'TeX',
       'Jupyter Notebook', 'Smarty', 'HTML', 'Yacc', 'Ruby', 'mcfunction',
       'HCL', 'Assembly', 'ZenScript', 'GLSL', 'JSON', 'Swift', 'Vue',
       'SCSS', 'AutoHotkey', 'Visual Basic .NET', 'Elixir', 'GDScript',
       'CMake', 'R', 'Haskell'

In [11]:
df['other'] = df[df['language']] != (df[df['language']] =='Java' | 'JavaScript' | 'Python')

KeyError: "None of [Index(['Python', 'Shell', 'Python', 'Java', 'Kotlin', 'C', 'Java', 'Shell',\n       'C#', 'C#',\n       ...\n       'TypeScript', 'Java', 'Rust', 'Python', 'JavaScript', 'Java', 'Java',\n       'Java', 'JavaScript', 'JavaScript'],\n      dtype='object', length=884)] are in the [columns]"

In [ ]:
df['language'] = np.where(df['language'] >= 22000, 15000, df['Fee'])